In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.13.1


In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Insepcting the Data

In [0]:
movie_data = pd.read_csv("gdrive/My Drive/movies_text.csv")

In [0]:


movie_data.shape
movie_data = movie_data.dropna()
movie_data = movie_data.reset_index(drop=True)
#movie_data.shape

In [0]:
movie_data.shape
movie_data=movie_data.rename(columns = {"Summary": "tagLine","Text":"movie_plot"})
movie_data = pd.DataFrame(movie_data, columns=["movie_plot","tagLine"])

In [0]:
print("Actual Data :",movie_data.shape)
print("Train data:",movie_data.iloc[:int(movie_data.shape[0]*0.6)].shape)
print("Validation Data:",movie_data.iloc[int(movie_data.shape[0]*0.6):int(movie_data.shape[0]*0.8)].shape)
print("Test Data:",movie_data.iloc[int(movie_data.shape[0]*0.80):].shape)


Actual Data : (20404, 2)
Train data: (12242, 2)
Validation Data: (4081, 2)
Test Data: (4081, 2)


In [0]:
movie_training_data=movie_data.iloc[:int(movie_data.shape[0]*0.6)].reset_index(drop=True)
movie_val_data=movie_data.iloc[int(movie_data.shape[0]*0.6):int(movie_data.shape[0]*0.8)].reset_index(drop=True)
movie_test_data=movie_data.iloc[int(movie_data.shape[0]*0.85):].reset_index(drop=True)
   

movie_training_data = movie_training_data.sample(frac=1).reset_index(drop=True)
movie_val_data = movie_val_data.sample(frac=1).reset_index(drop=True)
movie_test_data = movie_test_data.sample(frac=1).reset_index(drop=True)

#print(movie_training_data.shape)
#print(movie_val_data.shape)
#print(movie_test_data.shape)

movie_data.head()


,movie_plot,tagLine
0,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!
1,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...
2,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...
3,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...
4,"Obsessive master thief, Neil McCauley leads a ...",A Los Angeles Crime Saga


In [0]:
# Inspecting Data
for i in range(10):
    print("Before Data Cleaning movie_Plot #",i+1)
    print(movie_training_data.movie_plot[i])
    print("Tagline")
    print(movie_training_data.tagLine[i])
    
    print()

Before Data Cleaning movie_Plot # 1
A battle ensues among two government spy teams in an underground facility after their boss is assassinated.
Tagline
Two rival teams of assassins. One killer day at the office.

Before Data Cleaning movie_Plot # 2
Bettie Page grew up in a conservative religious family in Tennessee and became a photo model sensation in 1950s New York. Bettie's legendary pin-up photos made her the target of a Senate investigation into pornography, and transformed her into an erotic icon who continues to enthrall fans to this day.
Tagline
The Pin-Up Sensation That Shocked The Nation.

Before Data Cleaning movie_Plot # 3
Two crew members wake up on an abandoned spacecraft with no idea who they are, how long they've been asleep, or what their mission is. The two soon discover they're actually not alone – and the reality of their situation is more horrifying than they could have imagined.
Tagline
Don't fear the end of the world. Fear what happens next.

Before Data Cleaning

**Data Wrangling**

In [0]:
# Reference for  list of contractions in English took from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [0]:
#Cleaning the data
def clean_text(text, remove_stopwords = True):
    
    
    
    # Convert words to lower case
    text = text.lower()
    
    # using contractions to replace short with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Extraction of unimportant characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    #removing stop words(optional if not nltk library)
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [0]:
#nltk library for removing Stop words
import nltk
#nltk.download()
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
# Clean the Training Taglines and Movie Plots 
clean_training_plot = []
for movieplot_train in movie_training_data.movie_plot:
    clean_training_plot.append(clean_text(movieplot_train))
print("Training movie plots are complete.")


clean_training_tag = []
for tagline_train in movie_training_data.tagLine:
    clean_training_tag.append(clean_text(tagline_train, remove_stopwords=False))
print("Training Taglines are complete.")




# Clean the Validation Taglines and Movie Plots 
clean_val_plot = []
for movieplot_val in movie_val_data.movie_plot:
    clean_val_plot.append(clean_text(movieplot_val))
print("Validation movie plots are complete.")

clean_val_tag = []
for tagline_val in movie_val_data.tagLine:
    clean_val_tag.append(clean_text(tagline_val, remove_stopwords=False))
print("Validation Taglines are complete.")




# Clean the Testing Taglines and Movie Plots 
clean_test_plot = []
for movieplot_test in movie_test_data.movie_plot:
    clean_test_plot.append(clean_text(movieplot_test))
print("Testing movie plots are complete.")



clean_test_tag = []
for tagline_test in movie_test_data.tagLine:
    clean_test_tag.append(clean_text(tagline_test, remove_stopwords=False))
print("Testing Taglines are complete.")



Training movie plots are complete.
Training Taglines are complete.
Validation movie plots are complete.
Validation Taglines are complete.
Testing movie plots are complete.
Testing Taglines are complete.


In [0]:
# Inspect the cleaned summaries and texts to ensure they have been cleaned well
for i in range(10):
    print("After Data Cleaning Movie_Plot #",i+1)
    print(clean_training_plot[i])
    print("Tagline")
    print(clean_training_tag[i])
    
    print()

After Data Cleaning Movie_Plot # 1
battle ensues among two government spy teams underground facility boss assassinated
Tagline
two rival teams of assassins  one killer day at the office 

After Data Cleaning Movie_Plot # 2
bettie page grew conservative religious family tennessee became photo model sensation 1950s new york bettie legendary pin photos made target senate investigation pornography transformed erotic icon continues enthrall fans day
Tagline
the pin up sensation that shocked the nation 

After Data Cleaning Movie_Plot # 3
two crew members wake abandoned spacecraft idea long asleep mission two soon discover actually alone – reality situation horrifying could imagined
Tagline
do not fear the end of the world  fear what happens next 

After Data Cleaning Movie_Plot # 4
charlie immigrant endures challenging voyage gets trouble soon arrives america
Tagline
 the tramp  arrives in new york

After Data Cleaning Movie_Plot # 5
story beautiful proud tough loner sailor named querelle w

In [0]:
#count of words
def words_counter(count_d, text):
    
    for sentence in text:
        for word in sentence.split():
            if word not in count_d:
                count_d[word] = 1
            else:
                count_d[word] += 1

In [0]:
#the size of the vocabulary
word_counts = {}

words_counter(word_counts, clean_training_tag)
words_counter(word_counts, clean_training_plot)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 37183


In [0]:
#For Language Embedding we are using Refernce from Conceptnet Numberbatch's (CN) embeddings

# (https://github.com/commonsense/conceptnet-numberbatch)
embeddings_index = {}
with open('gdrive/My Drive/numberbatch-en.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))
#embeddings_index.shape

Word embeddings: 417195


In [0]:
# Find missing words from from conceptnet-numberbatch, and are used more than our threshold.
missing_words = 0
threshold = 5

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round((1.0*missing_words/len(word_counts))*100,4)

print("Number of words missing from CN:", missing_words)
#print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 184


In [0]:

#converting words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

#tokens that will be used to our vocab and for seqtoseq model 
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
#print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 37183
Number of words we will use: 32311


In [0]:
#embedding dimensions to match conceptnet-numberbatch.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Default matrix with zeroes
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in conceptnet-numberbatch, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

32311


In [0]:

'''Convert words in Movie_plot to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Count the number of words and UNKs.
       Adding  EOS token to the end of texts for seq2seq model'''
def convert_to_ints(text, word_count, unk_count, eos=False):
    
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [0]:
# Apply convert_to_ints to clean_Movie_plot and clean_taglines
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_training_tag, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_training_plot, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in Movie_plot:", word_count)
print("Total number of UNKs in Movie_plot:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in Movie_plot: 495597
Total number of UNKs in Movie_plot: 5999
Percent of words that are UNK: 1.21%


In [0]:
#DataFrame
def create_lengths(text):
    
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [0]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

#print("Summaries:")
#print(lengths_summaries.describe())
#print()
#print("Texts:")
#print(lengths_texts.describe())

In [0]:
'''Counter for UNK appears in a sentence.'''
def unk_counter(sentence):
    
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [0]:
# Sorting the Movie_plot and Taglines by the length of the texts, shortest to longest
# Limiting the length of Movie_plot and Tagline based Min-Max range.
# Remove Taglines that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 200
max_summary_length = 50
min_length = 2
unk_text_limit = 1
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))
print(type(sorted_summaries))
print(type(sorted_texts))
print(sorted_summaries[:10])
print(sorted_texts[:10])




10664
10664
<class 'list'>
<class 'list'>
[[540, 636, 540], [67, 439, 38, 1597, 1598], [9, 128, 3, 9, 392, 3, 305], [4443, 2884, 9, 7, 9, 21, 1074], [191, 296, 35, 297, 64, 113, 67, 45, 711], [63, 3, 1591, 794, 1150], [9, 8511, 277, 9, 8511, 3760, 9, 8511, 250], [99, 31, 859, 33, 27, 3092, 384, 458, 384, 109, 708], [490, 527, 4752, 33, 17, 18, 948, 81, 709, 22, 490, 527, 9, 3697, 948, 191, 948, 490, 527, 27, 67, 108, 35, 2210, 195, 35, 147, 379, 1520, 1841, 858, 371, 35, 360, 126, 1800], [9, 30, 126, 1081, 636, 57, 160, 436, 183, 513, 82, 573, 3, 329]]
[[12236, 798, 32309], [12236, 798, 32309], [128, 392, 305, 32309], [1519, 7991, 1295, 32309], [3684, 67, 287, 32309], [1591, 794, 1150, 32309], [1704, 8511, 1601, 32309], [16035, 1635, 1699, 1118, 32309], [360, 2387, 236, 566, 32309], [542, 1292, 1832, 573, 32309]]


## Creation of the Tagline prediction Model

In [0]:
def model_inputs():
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [0]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [0]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    
    
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [0]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    
    
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits, _ , _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_decoder

In [0]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, _ , _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_decoder

In [0]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    
    
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
            
    
    initial_state = dec_cell.zero_state(batch_size=batch_size,dtype=tf.float32).clone(cell_state=enc_state[0])

    with tf.variable_scope("decode"):
        training_decoder = training_decoding_layer(dec_embed_input, 
                                                  summary_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_summary_length)
        
        training_logits,_ ,_ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                  output_time_major=False,
                                  impute_finished=True,
                                  maximum_iterations=max_summary_length)
    with tf.variable_scope("decode", reuse=True):
        inference_decoder = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)
        
        inference_logits,_ ,_ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                  output_time_major=False,
                                  impute_finished=True,
                                  maximum_iterations=max_summary_length)

    return training_logits, inference_logits

In [0]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    
    
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [0]:
def pad_sentence_batch(sentence_batch):
    
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [0]:
def get_batches(summaries, texts, batch_size):
    
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        
        
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [0]:
#Hyper parameters
epochs = 30
batch_size = 64
rnn_size = 256
num_layers = 4
learning_rate = 0.002
keep_probability = 0.80

In [0]:
import warnings
warnings.filterwarnings("ignore")


In [0]:
import warnings
warnings.filterwarnings("ignore")
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

Graph is built.


## Training the Model

In [0]:
# Subset of dataset for training
start =0
end = start + 5000
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("Movie_plot shortest subset:", len(sorted_texts_short[0]))
print("The longest length:",len(sorted_texts_short[-1]))

Movie_plot shortest subset: 3
The longest length: 27


In [0]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 
stop_early = 0 
stop = 3 
per_epoch = 3 
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] 

checkpoint = "best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
   
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print("Building the Model...")
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                
                
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Building the Model...
Epoch   1/30 Batch   20/78 - Loss:  3.395, Seconds: 5.43
Average loss for this update: 3.195
New Record!
Building the Model...
Epoch   1/30 Batch   40/78 - Loss:  2.357, Seconds: 8.69
Average loss for this update: 2.41
New Record!
Building the Model...
Epoch   1/30 Batch   60/78 - Loss:  2.510, Seconds: 5.82
Average loss for this update: 2.502
No Improvement.
Building the Model...
Epoch   2/30 Batch   20/78 - Loss:  2.219, Seconds: 5.36
Average loss for this update: 2.212
New Record!
Building the Model...
Epoch   2/30 Batch   40/78 - Loss:  2.186, Seconds: 8.56
Average loss for this update: 2.259
No Improvement.
Building the Model...
Epoch   2/30 Batch   60/78 - Loss:  2.375, Seconds: 5.93
Average loss for this update: 2.381
No Improvement.
Building the Model...
Epoch   3/30 Batch   20/78 - Loss:  2.173, Seconds: 5.28
Average loss for this update: 2.162
New Record!
Building the Model...
Epoch   3/30 Batch   40/78 - Loss:  2.114, Seconds: 8.55
Average loss for this

In [0]:
import datetime
st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
print(st)

2019-04-29 10:29:58


**Train** **data**

In [0]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [0]:
output_summaries = []
checkpoint = "./best_model.ckpt"
pad = vocab_to_int["<PAD>"] 
loaded_graph = tf.Graph()
clean_train_plot_top = clean_training_plot[0:10]
clean_train_tag_top= clean_training_tag[0:10]

with tf.Session(graph=loaded_graph) as sess:
    
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    
    for i in range(0, len(clean_train_tag_top10)):
        text = text_to_seq(clean_train_plot_top10[i])
        answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(10,20)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 
        temp = " ".join([int_to_vocab[i] for i in answer_logits if i != pad])
        output_summaries.append(temp)
        

INFO:tensorflow:Restoring parameters from ./best_model.ckpt


In [0]:
!pip install rouge

In [0]:
import json
from rouge import Rouge

In [0]:
rouge = Rouge()
scores = rouge.get_scores(output_summaries, clean_train_tag_top10, avg = True)
print(st)
print("Training data scores:",scores)

2019-04-29 10:29:58
Training data scores: {'rouge-1': {'f': 0.24500500883292103, 'p': 0.32107142857142856, 'r': 0.20662679425837321}, 'rouge-2': {'f': 0.11742424051143381, 'p': 0.15194805194805192, 'r': 0.09636363636363637}, 'rouge-l': {'f': 0.2202135947193904, 'p': 0.32107142857142856, 'r': 0.20662679425837321}}


**Validation** ***Data***

In [0]:
print("Datetime:",st)
output_summaries = []
checkpoint = "./best_model.ckpt"
pad = vocab_to_int["<PAD>"] 
loaded_graph = tf.Graph()
clean_val_plot_top10 = clean_training_plot[0:20]
clean_val_tag_top10 = clean_training_tag[0:20] 
with tf.Session(graph=loaded_graph) as sess:
    # Loading model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    
    for i in range(0, len(clean_val_tag_top10)):
      
        text = text_to_seq(clean_val_plot_top10[i])
        answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(10,20)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 
        temp = " ".join([int_to_vocab[i] for i in answer_logits if i != pad])
        output_summaries.append(temp)
       

Datetime: 2019-04-29 10:29:58
INFO:tensorflow:Restoring parameters from ./best_model.ckpt


In [0]:
scores = rouge.get_scores(output_summaries, clean_val_tag_top10, avg = True)
print("Validation Data: ",(scores))

Validation Data:  {'rouge-1': {'f': 0.20753608066197904, 'p': 0.29148809523809527, 'r': 0.16827551834130783}, 'rouge-2': {'f': 0.08459447274928093, 'p': 0.11097402597402595, 'r': 0.06901515151515152}, 'rouge-l': {'f': 0.17636395692073467, 'p': 0.27898809523809526, 'r': 0.16327551834130782}}


**Test Data**

In [0]:
print("Datetime:",st)
output_summaries = []
checkpoint = "./best_model.ckpt"
pad = vocab_to_int["<PAD>"] 
loaded_graph = tf.Graph()
clean_test_plot_top10 = clean_training_plot[0:50]
clean_test_tag_top10 = clean_training_tag[0:50] 
with tf.Session(graph=loaded_graph) as sess:
    # Loading  saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    
    for i in range(0, len(clean_test_tag_top10)):
      
        text = text_to_seq(clean_test_plot_top10[i])
        answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(10,20)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 
        temp = " ".join([int_to_vocab[i] for i in answer_logits if i != pad])
        output_summaries.append(temp)
        

Datetime: 2019-04-29 10:29:58
INFO:tensorflow:Restoring parameters from ./best_model.ckpt


In [0]:
scores = rouge.get_scores(output_summaries, clean_test_tag_top10, avg = True)
print("Test Data:",scores)

Test Data: {'rouge-1': {'f': 0.19861789291040188, 'p': 0.2636587301587301, 'r': 0.18509125549373226}, 'rouge-2': {'f': 0.08321033759052515, 'p': 0.09755627705627705, 'r': 0.08081118881118882}, 'rouge-l': {'f': 0.16037128721672184, 'p': 0.24149206349206342, 'r': 0.17064147488218698}}


# **Tagline Predictions**

In [0]:

#input_sentence = "Hi Surekha.I love irobot movie and the plot is as follows.A robot may not injure a human being or, through inaction, allow a human being to come to harm"
#text = text_to_seq(input_sentence)
random = np.random.randint(0,len(clean_training_plot))
input_sentence = clean_training_plot[random]
text = text_to_seq(clean_training_plot[random])

checkpoint = "./best_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Loading saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(5,8)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 


pad = vocab_to_int["<PAD>"] 

print('Original Text:', movie_training_data.movie_plot[random])
print('Original summary:', movie_training_data.tagLine[random])
print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from ./best_model.ckpt
Original Text: Terry is a suicidal voyeur who treats a dying addict to a final binge, but Terry will only do this if he promises to kill him.
Original summary: An oddball odyssey about voyeurism, LSD and nude bowling!

Text
  Word Ids:    [14588, 4703, 14607, 3685, 1625, 1715, 2302, 19235, 14588, 5330, 778]
  Input Words: terry suicidal voyeur treats dying addict final binge terry promises kill

Summary
  Word Ids:       [205, 57, 58, 438, 58, 300]
  Response Words: nothing is no shot no game


**Genalization of our Model**

In [0]:
# Create your own review or use one from the dataset
#input_sentence = "Robbie, is set in 1998 and centres on a little girl, Gloria, who loves her nursemaid robot, Robbie. Her mother comes to believe that robots are unsafe, however, and Robbie is returned to the factory. Gloria is heartbroken. In an effort to show her that robots are machines, not people, her parents take her to see robots being assembled at a factory. One of the assembling robots is Robbie. Gloria endangers her life running to Robbie, and Robbie rescues Gloria, persuading Gloria's mother that robots can be trusted"
title = "A Revenge Story"
input_sentence="When Bhallaladeva conspires against his brother to become the king of Mahishmati, he has him killed by Katappa and imprisons his wife. Years later, his brother's son returns to avenge his father's death."
text = text_to_seq(input_sentence)
#random = np.random.randint(0,len(clean_training_plot))
#input_sentence = clean_training_plot[random]
#text = text_to_seq(clean_training_plot[random])

checkpoint = "./best_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
   
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(5,8)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 


pad = vocab_to_int["<PAD>"] 

print('Original Title:', title)

print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from ./best_model.ckpt
Original Title: A Revenge Story

Text
  Word Ids:    [32307, 16640, 1309, 779, 392, 32307, 2062, 32307, 29594, 1271, 1151, 3178, 1309, 449, 1832, 3916, 1645, 500]
  Input Words: <UNK> conspires brother become king <UNK> killed <UNK> imprisons wife years later brother son returns avenge father death

Summary
  Word Ids:       [9, 341, 9, 341, 9, 367]
  Response Words: the suspense the suspense the truth


**Detecting Mistakes in predicting Taglines**

In [0]:

input_sentence = "Hi This is Surekha from Hyderabad.I love irobot movie and the plot is as follows.A robot may not injure a human being or, through inaction, allow a human being to come to harm"
text = text_to_seq(input_sentence)

checkpoint = "./best_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(5,8)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 


pad = vocab_to_int["<PAD>"] 


print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from ./best_model.ckpt

Text
  Word Ids:    [3851, 32307, 32307, 67, 32307, 104, 2119, 2098, 935, 857, 32307, 1173, 32307, 6289, 1173, 79, 3714]
  Input Words: hi <UNK> <UNK> love <UNK> movie plot follows robot may <UNK> human <UNK> allow human come harm

Summary
  Word Ids:       [9, 128, 3, 9, 21]
  Response Words: the story of the world


Mistakes:1)Names are becoming unownk for local names and places
                  2)Repetation of words 
                  3)Not exact prediction because restriction on Tagline length and Plot
                  4)Vocab Mistakes because of human written data